Scraping pagina Web Colombia TIC

## Análisis de Reportes de Colombia TIC

Este notebook permite automatizar la descarga de reportes de Colombia TIC, extraer y procesar datos relevantes, 
y finalmente combinar la información en un solo DataFrame para análisis.

In [1]:
# librerias necesarias para el proyecto

import os
import re
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

## Configuración Inicial

In [2]:
# Configuración de la URL base para la descarga y el uso de carpetas de almacenamiento.

BASE_URL = "https://colombiatic.mintic.gov.co/679/w3-channel.html"
download_base_path = "reporteColombiaTic"
combined_data_path = "reporteCombinado"

# ### Configuración de Selenium WebDriver

# Configurar el servicio de Chrome con la ruta completa
service = Service("C:/Users/Josvaldes/Documents/Maestria/Austral/trabajoGrado/trabajoGradoMCD/Codigo/chromedriver.exe")

# Opciones de Chrome (opcional, para correr en modo headless)
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

try:
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(BASE_URL)
    print("Título de la página de prueba:", driver.title)
except Exception as e:
    print("Error al inicializar ChromeDriver:", e)
finally:
    driver.quit()

Título de la página de prueba: Inicio


## Funciones

In [3]:
# ### Función para crear subcarpetas

def create_directories(base_path, year, trimester):
    """
    Crea directorios para almacenar los archivos según año y trimestre.
    """
    path = os.path.join(base_path, f"Año_{year}", f"Trimestre_{trimester}")
    os.makedirs(path, exist_ok=True)
    return path

# ### Función para descargar el reporte

def download_report(url, save_path):
    """
    Descarga el archivo Excel desde la URL proporcionada y lo guarda en el path especificado.
    """
    response = requests.get(url)
    filename = os.path.join(save_path, "reporte_tic_trimestral.xlsx")
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f"Reporte descargado en {filename}")
    return filename

# ## Descarga y Almacenamiento del Reporte

def archivo_trimestral_existente(nombre_archivo):
    return os.path.exists(nombre_archivo)

# Función principal para descargar el reporte
# Definir la ruta de la carpeta donde se guardará el archivo descargado
download_base_path = "reporteColombiaTic"

def fetch_report(url):
    """
    Navega en la página de Colombia TIC, encuentra y descarga el archivo Excel más reciente.
    """
    # Configurar el servicio de Chrome
    service = Service('C:/Users/Josvaldes/Documents/Maestria/Austral/trabajoGrado/trabajoGradoMCD/Codigo/chromedriver.exe')
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=service, options=options)
    
    driver.get(url)

    try:
        # Esperar y encontrar la sección "Destacados"
        destacados_header = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//h2[@class='ntg-titulo-caja' and text()='Destacados']"))
        )
        print("Se encontró la sección 'Destacados'.")

        # Buscar el enlace al artículo en la sección "Destacados"
        destacados_section = driver.find_element(By.XPATH, "//div[@class='recuadro col-md-6 col-lg-4 text-center mb-4']")
        link_element = destacados_section.find_element(By.TAG_NAME, "a")
        report_url = link_element.get_attribute('href')

        print("URL del artículo:", report_url)

        # Extraer el trimestre y año del reporte a partir del texto del enlace
        trimestre_texto = link_element.text
        trimestre = re.search(r"(primer|segundo|tercer|cuarto) trimestre de (\d{4})", trimestre_texto, re.IGNORECASE)
        if trimestre:
            trimestre_nombre = trimestre.group(1)
            año = trimestre.group(2)
            nombre_archivo = f"reporte_tic_{trimestre_nombre}_trimestre_{año}.xlsx"
        else:
            print("No se pudo determinar el trimestre del reporte.")
            nombre_archivo = "reporte_tic_trimestral.xlsx"

        # Crear la carpeta de descarga si no existe
        os.makedirs(download_base_path, exist_ok=True)
        
        # Ruta completa donde se guardará el archivo
        archivo_path = os.path.join(download_base_path, nombre_archivo)
        
        # Verificar si el archivo del trimestre ya existe
        if os.path.exists(archivo_path):
            print(f"El archivo '{archivo_path}' ya existe. No se realizará la descarga.")
            return archivo_path
        else:
            # Navegar a la página del artículo
            driver.get(report_url)

            # Esperar a que el `<span>` que contiene el enlace al archivo Excel esté presente
            excel_span = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//span[contains(@class, 'bajardoc') and contains(@class, 'binary-archivo_xls') and contains(@class, 'format-xlsx')]"))
            )

            # Obtener el enlace al archivo Excel
            excel_link = excel_span.find_element(By.TAG_NAME, "a")
            excel_url = excel_link.get_attribute('href')
            print("URL del archivo Excel:", excel_url)

            # Descargar el archivo Excel usando requests
            excel_response = requests.get(excel_url)

            # Verificar el código de estado antes de guardar
            if excel_response.status_code == 200:
                with open(archivo_path, 'wb') as file:
                    file.write(excel_response.content)
                print(f"Reporte descargado exitosamente como {archivo_path}")
                return archivo_path
            else:
                print(f"Error al descargar el archivo. Código de estado: {excel_response.status_code}")
                return None

    except Exception as e:
        print("No se encontró la sección 'Destacados' o hubo un error:", str(e))
        return None

    finally:
        # Cerrar el navegador
        driver.quit()


# Función para renombrar columnas duplicadas con sufijos secuenciales
def rename_duplicates(columns):
    """
    Renombra columnas duplicadas añadiendo un sufijo '_dupN' para evitar conflictos de nombres.
    """
    counts = {}
    new_columns = []
    for col in columns:
        if col in counts:
            counts[col] += 1
            new_columns.append(f"{col}_dup{counts[col]}")
        else:
            counts[col] = 0
            new_columns.append(col)
    return new_columns

def save_large_df_to_excel(df, file_path, chunk_size=1000000):
    """
    Guarda un DataFrame grande en un archivo Excel, dividiéndolo en múltiples hojas si es necesario.
    Args:
        df (DataFrame): DataFrame a guardar.
        file_path (str): Ruta del archivo Excel de salida.
        chunk_size (int): Número máximo de filas por hoja.
    """
    with pd.ExcelWriter(file_path) as writer:
        for i in range(0, len(df), chunk_size):
            df_chunk = df.iloc[i:i + chunk_size]
            sheet_name = f"Sheet_{i // chunk_size + 1}"
            df_chunk.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"Archivo combinado final guardado en {file_path}")

def save_large_df_to_csv(df, base_path, chunk_size=1_000_000):
    """
    Guarda un DataFrame grande en múltiples archivos CSV en una subcarpeta especificada.
    
    Parámetros:
    - df: DataFrame a guardar.
    - base_path: Ruta base para guardar los archivos CSV.
    - chunk_size: Tamaño del lote en número de filas para dividir el archivo CSV si es grande.
    """
    # Asegurarse de que la carpeta base exista
    os.makedirs(base_path, exist_ok=True)

    # Dividir y guardar el DataFrame en archivos CSV en chunks
    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i:i + chunk_size]
        file_path = os.path.join(base_path, f"combined_data_part_{i // chunk_size + 1}.csv")
        chunk.to_csv(file_path, index=False)
        print(f"Guardado el archivo {file_path}")

# ## Procesamiento del Reporte

def process_report2(file_path):
    """
    Procesa cada hoja del archivo descargado, aplica limpieza y estandarización de columnas.
    """
    # Definir las columnas esperadas para cada hoja
    column_structure = {
        '1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'No. ACCESOS FIJOS A INTERNET'],
        '2': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
        '3': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
        '4,1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,2': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,3': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,4': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '5': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS (Pesos)'],
        '6': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. ABONADOS'],
        '7': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
        '8': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TRÁFICO (MB)'],
        '9': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. SUSCRIPTORES'],
        '10': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
        '11': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'TRÁFICO (MB)'],
        '12': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'LÍNEAS EN SERVICIO', 'LÍNEAS PREPAGO', 'LÍNEAS POSPAGO', 'LÍNEAS ACTIVADAS', 'LÍNEAS RETIRADAS'],
        '13': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'PROVEEDOR DESTINO', 'TRÁFICO PREPAGO', 'TRÁFICO POSPAGO'],
        '14': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CONSUMO PREPAGO', 'CONSUMO POSPAGO', 'INGRESOS OPERACIONALES'],
        '15': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'CABECERA MUNICIPAL', 'CÓDIGO DANE', 'CENTRO POBLADO', 'COBERTURA 2G', 'COBERTURA 3G', 'COBERTURA HSPA+', 'HSPA+DC', 'COBERTURA_4G', 'COBERTURA_LTE', 'COBERTURA_5G'],
        '16': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE DEPARTAMENTO', 'DEPARTAMENTO', 'CÓDIGO DANE MUNICIPIO', 'MUNICIPIO', 'SEGMENTO', 'LÍNEAS EN SERVICIO'],
        '17': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS TRÁFICO LDIE', 'INGRESOS TRÁFICO LDIS', 'INGRESOS TELEFONIA FIJA']
    }

    all_sheets = pd.read_excel(file_path, sheet_name=None)
    processed_sheets = {}
    
    # Función para renombrar columnas duplicadas
    def rename_duplicates(columns):
        counts = {}
        new_columns = []
        for col in columns:
            if col in counts:
                counts[col] += 1
                new_columns.append(f"{col}_dup{counts[col]}")
            else:
                counts[col] = 0
                new_columns.append(col)
        return new_columns

    # Limpiar cada hoja y ajustar las columnas
    for sheet_name, df in all_sheets.items():
        if sheet_name not in column_structure:
            print(f"Omitiendo la hoja '{sheet_name}' (sin estructura conocida)")
            continue

        # Limpiar encabezados y resetear índices
        df = df.rename(columns=lambda x: x.strip()).dropna(how='all')
        
        # Renombrar columnas duplicadas
        df.columns = rename_duplicates(df.columns)

        # Validar si el número de columnas coincide
        if len(df.columns) == len(column_structure[sheet_name]):
            df.columns = column_structure[sheet_name]
            processed_sheets[sheet_name] = df
        else:
            print(f"Omitiendo la hoja '{sheet_name}' debido a la longitud de columnas no coincidente.")
    
    # Guardar un archivo temporal para depuración
    temp_output_path = os.path.join("reporteCombinado", "temp_combined_debug.xlsx")
    with pd.ExcelWriter(temp_output_path) as writer:
        for sheet_name, df in processed_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"Archivo temporal guardado en: {temp_output_path}")

    # Combinar todas las hojas en un solo DataFrame
    combined_df = processed_sheets.pop('1')
    for sheet_name, df in processed_sheets.items():
        # Verificar y renombrar columnas duplicadas que podrían causar problemas en el merge
        for col in df.columns:
            if col in combined_df.columns and col != 'AÑO' and col != 'TRIMESTRE':
                df = df.rename(columns={col: f"{col}_{sheet_name}"})

        # Obtener columnas comunes
        common_columns = combined_df.columns.intersection(df.columns).tolist()
        combined_df = pd.merge(combined_df, df, on=common_columns, how='outer', suffixes=('', f'_{sheet_name}'))

    return combined_df


def process_report3(file_path, temp_folder="reporteCombinado/temp_lotes", min_column_count=5, batch_size=10):
    """
    Procesa cada hoja de un archivo Excel, la guarda en archivos temporales por lotes y luego combina los resultados.
    Args:
        file_path (str): Ruta del archivo Excel a procesar.
        temp_folder (str): Carpeta para guardar los archivos procesados temporalmente.
        min_column_count (int): Número mínimo de columnas para procesar una hoja.
        batch_size (int): Número de archivos en cada lote para la combinación por lotes.
    Returns:
        DataFrame: DataFrame combinado de todas las hojas procesadas.
    """
    # Crear la carpeta temporal para guardar archivos procesados
    os.makedirs(temp_folder, exist_ok=True)
    
    # Cargar el archivo Excel
    excel_file = pd.ExcelFile(file_path)
    
    # Paso 1: Procesa cada hoja y guarda el resultado individualmente
    for sheet_name in excel_file.sheet_names:
        try:
            # Leer la hoja
            df = excel_file.parse(sheet_name)
            
            # Verificar la estructura de la hoja antes de procesar
            if len(df.columns) < min_column_count:
                print(f"Omitiendo la hoja '{sheet_name}' debido a longitud de columnas insuficiente.")
                continue
            
            # Limpiar los nombres de las columnas y renombrar duplicados
            df.columns = [col.strip() for col in df.columns]  # Quitar espacios en blanco
            df.columns = rename_duplicates(df.columns)
            
            # Guardar cada hoja procesada en un archivo temporal CSV
            temp_file_path = os.path.join(temp_folder, f"{sheet_name}_processed.csv")
            df.to_csv(temp_file_path, index=False)
            print(f"Hoja '{sheet_name}' procesada y guardada en {temp_file_path}")

        except Exception as e:
            print(f"Error al procesar la hoja '{sheet_name}': {e}")

    # Paso 2: Combina los archivos temporales en lotes
    # Lista de archivos temporales procesados
    temp_files = [os.path.join(temp_folder, f) for f in os.listdir(temp_folder) if f.endswith('_processed.csv')]

    # DataFrame combinado final
    combined_df = pd.DataFrame()

    # Combinación por lotes
    for i in range(0, len(temp_files), batch_size):
        batch_files = temp_files[i:i + batch_size]
        
        # Leer y combinar el lote actual
        batch_df = pd.concat([pd.read_csv(f) for f in batch_files], axis=0, ignore_index=True)
        
        # Combina el lote con el DataFrame final
        if not combined_df.empty:
            common_columns = combined_df.columns.intersection(batch_df.columns).tolist()
            combined_df = pd.merge(combined_df, batch_df, on=common_columns, how='outer', suffixes=('', '_dup'))
        else:
            combined_df = batch_df.copy()  # Inicializar con el primer lote
        
        print(f"Lote {i // batch_size + 1} combinado exitosamente.")

    return combined_df

# ## Guardar el DataFrame Combinado

def save_combined_data(df, path):
    """
    Guarda el DataFrame combinado en un archivo Excel en la carpeta 'reporteCombinado'.
    """
    output_path = os.path.join(path, "data_reporte_combined_final.xlsx")
    os.makedirs(path, exist_ok=True)
    df.to_excel(output_path, index=False)
    print(f"DataFrame combinado guardado en: {output_path}")



# ## Análisis y Validación de Datos

# ### Función para verificar valores únicos y posibles inconsistencias

def analyze_columns(df):
    """
    Muestra valores únicos y verifica consistencias en las columnas clave.
    """
    columnas_clave = ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'DEPARTAMENTO', 'No. ACCESOS FIJOS A INTERNET']
    
    # Mostrar valores únicos en columnas clave
    for col in columnas_clave:
        if col in df.columns:
            unique_values = df[col].unique()
            print(f"\nValores únicos en '{col}':", unique_values[:10])  # Muestra los primeros 10 valores únicos

# ### Función para identificar y manejar datos nulos

def handle_missing_values(df):
    """
    Identifica y maneja valores nulos en el DataFrame, aplicando reemplazos si es necesario.
    """
    missing_info = df.isnull().sum()
    print("\nValores nulos en cada columna:\n", missing_info)
    
    # Puedes aplicar reemplazos según lo necesario, por ejemplo, reemplazar NaN en algunas columnas
    df['No. ACCESOS FIJOS A INTERNET'] = df['No. ACCESOS FIJOS A INTERNET'].fillna(0)
    # Más reemplazos pueden añadirse aquí según los análisis
    return df

# ### Función para generar estadísticas descriptivas

def generate_statistics(df):
    """
    Genera estadísticas descriptivas para las columnas numéricas del DataFrame.
    """
    numeric_cols = df.select_dtypes(include=['number']).columns
    print("\nEstadísticas descriptivas de columnas numéricas:\n", df[numeric_cols].describe())

# ### Guardar versión final del DataFrame para análisis

def save_final_data(df):
    """
    Guarda el DataFrame limpio y validado en un archivo Excel en la subcarpeta /reporteCombinado para análisis final.
    """
    final_output_path = os.path.join(combined_data_path, "data_reporte_combined_validated.xlsx")
    df.to_excel(final_output_path, index=False)
    print(f"DataFrame final validado guardado en: {final_output_path}")


# Carpeta para los archivos procesados temporalmente
temp_folder = "reporteCombinado/temp_lotes"
os.makedirs(temp_folder, exist_ok=True)

# Mínimo número de columnas esperadas para cada hoja (ajustar según tu estructura)
cierto_numero_de_columnas_minimo = 4

# Estructura esperada de columnas por hoja (añadir según tu estructura)
column_structure = {
    '1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'No. ACCESOS FIJOS A INTERNET'],
    '2': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
    '3': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
    '4,1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
    '4,2': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
    '4,3': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
    '4,4': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
    '5': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS (Pesos)'],
    '6': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. ABONADOS'],
    '7': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
    '8': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TRÁFICO (MB)'],
    '9': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. SUSCRIPTORES'],
    '10': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
    '11': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'TRÁFICO (MB)'],
    '12': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'LÍNEAS EN SERVICIO', 'LÍNEAS PREPAGO', 'LÍNEAS POSPAGO', 'LÍNEAS ACTIVADAS', 'LÍNEAS RETIRADAS'],
    '13': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'PROVEEDOR DESTINO', 'TRÁFICO PREPAGO', 'TRÁFICO POSPAGO'],
    '14': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CONSUMO PREPAGO', 'CONSUMO POSPAGO', 'INGRESOS OPERACIONALES'],
    '15': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'CABECERA MUNICIPAL', 'CÓDIGO DANE', 'CENTRO POBLADO', 'COBERTURA 2G', 'COBERTURA 3G', 'COBERTURA HSPA+', 'HSPA+DC', 'COBERTURA_4G', 'COBERTURA_LTE', 'COBERTURA_5G'],
    '16': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE DEPARTAMENTO', 'DEPARTAMENTO', 'CÓDIGO DANE MUNICIPIO', 'MUNICIPIO', 'SEGMENTO', 'LÍNEAS EN SERVICIO'],
    '17': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS TRÁFICO LDIE', 'INGRESOS TRÁFICO LDIS', 'INGRESOS TELEFONIA FIJA']
}
# Función para limpiar y estandarizar cada hoja
# Función para limpiar y estandarizar cada hoja
def limpiar_hoja(df, columnas_esperadas):
    """
    Limpia una hoja eliminando encabezados irrelevantes y ajustando las columnas.
    Args:
        df (DataFrame): Hoja leída del Excel.
        columnas_esperadas (list): Lista de columnas esperadas.
    Returns:
        DataFrame: Hoja limpiada.
    """
    # Eliminar filas completamente vacías
    df = df.dropna(how="all")
    
    # Intentar detectar la fila donde comienzan los datos relevantes
    valid_row_index = None
    for i, row in df.iterrows():
        if row.notna().sum() >= len(columnas_esperadas) - 2:  # Flexibilidad para detectar columnas faltantes
            valid_row_index = i
            break
    
    if valid_row_index is not None:
        # Cortar el DataFrame desde la fila detectada
        df = df.iloc[valid_row_index:].reset_index(drop=True)
        
        # Reasignar los nombres de las columnas a partir de las esperadas
        df.columns = columnas_esperadas[: len(df.columns)]  # Ajustar columnas al número detectado
        df = df.reset_index(drop=True)
    else:
        print(f"No se encontraron filas válidas en la hoja con columnas esperadas: {columnas_esperadas}.")
    
    return df

# Detectar todas las columnas únicas en el archivo
def detect_all_columns(file_path, column_structure):
    """
    Detecta todas las columnas únicas en un archivo Excel basándose en la estructura definida.
    Args:
        file_path (str): Ruta del archivo Excel.
        column_structure (dict): Estructura esperada de las columnas.
    Returns:
        list: Lista de todas las columnas únicas en el archivo.
    """
    all_columns = set()
    for sheet_name in pd.ExcelFile(file_path).sheet_names:
        try:
            # Verifica si la hoja está en la estructura esperada
            if sheet_name in column_structure:
                all_columns.update(column_structure[sheet_name])
            else:
                print(f"Omitiendo hoja '{sheet_name}' porque no está en la estructura.")
        except Exception as e:
            print(f"Error al procesar la hoja '{sheet_name}': {e}")
    return list(all_columns)

# Procesar y combinar reporte
def process_report(file_path, column_structure, temp_folder="reporteCombinado/temp_lotes"):
    """
    Procesa cada hoja del reporte, limpia y alinea columnas, y combina el resultado en un archivo CSV final.
    """
    # Detectar todas las columnas únicas basadas en column_structure
    all_columns = detect_all_columns(file_path, column_structure)
    os.makedirs(temp_folder, exist_ok=True)

    # Procesamiento y almacenamiento de cada hoja
    for sheet_name in pd.ExcelFile(file_path).sheet_names:
        if sheet_name in column_structure:
            try:
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                cleaned_df = limpiar_hoja(df, column_structure[sheet_name])
                
                # Guardar cada hoja procesada en un archivo temporal CSV
                temp_file_path = os.path.join(temp_folder, f"{sheet_name}_processed.csv")
                cleaned_df.to_csv(temp_file_path, index=False)
                print(f"Hoja '{sheet_name}' procesada y guardada en {temp_file_path}")
            except Exception as e:
                print(f"Error al procesar la hoja '{sheet_name}': {e}")
        else:
            print(f"Omitiendo la hoja '{sheet_name}' (estructura desconocida)")

    # Combinación de los archivos CSV temporales en un archivo final
    combined_data_path = "reporteCombinado/combined_data_final.csv"
    temp_files = [os.path.join(temp_folder, f) for f in os.listdir(temp_folder) if f.endswith('_processed.csv')]

    with open(combined_data_path, 'w', newline='', encoding='utf-8') as output_file:
        for i, temp_file in enumerate(temp_files):
            batch_df = pd.read_csv(temp_file, dtype=str, low_memory=False)
            batch_df = batch_df.reindex(columns=all_columns, fill_value="")  # Alinear todas las columnas
            
            # Concatenar archivos con encabezado solo en el primer archivo
            batch_df.to_csv(output_file, mode='a', header=(i == 0), index=False)
            print(f"Archivo {temp_file} concatenado exitosamente en {combined_data_path}")

    print(f"Archivo combinado final guardado en {combined_data_path}")



def process_report_con_hojas_4x(file_path, column_structure, temp_folder="reporteCombinado/temp_lotes"):
    """
    Procesa todas las hojas del archivo Excel, eliminando encabezados irrelevantes y ajustando las columnas.
    """
    os.makedirs(temp_folder, exist_ok=True)
    excel_file = pd.ExcelFile(file_path)

    for sheet_name in excel_file.sheet_names:
        try:
            # Leer la hoja
            df = pd.read_excel(file_path, sheet_name=sheet_name)

            # Verificar si la hoja está en la estructura esperada
            if sheet_name in column_structure:
                cleaned_df = limpiar_hoja(df, column_structure[sheet_name])
            else:
                print(f"Omitiendo hoja '{sheet_name}' porque no está en la estructura.")
                continue

            # Guardar la hoja procesada en un archivo temporal CSV
            temp_file_path = os.path.join(temp_folder, f"{sheet_name}_processed.csv")
            cleaned_df.to_csv(temp_file_path, index=False)
            print(f"Hoja '{sheet_name}' procesada y guardada en {temp_file_path}")

        except Exception as e:
            print(f"Error al procesar la hoja '{sheet_name}': {e}")


def process_report_con_hojas_4x(file_path, column_structure, temp_folder="reporteCombinado/temp_lotes"):
    """
    Procesa todas las hojas del archivo Excel aplicando limpieza especial para las hojas '4,x'.
    Args:
        file_path (str): Ruta del archivo Excel a procesar.
        column_structure (dict): Estructura esperada de las columnas.
        temp_folder (str): Carpeta donde se guardarán los archivos procesados temporalmente.
    """
    os.makedirs(temp_folder, exist_ok=True)
    excel_file = pd.ExcelFile(file_path)

    for sheet_name in excel_file.sheet_names:
        try:
            # Leer la hoja
            df = pd.read_excel(file_path, sheet_name=sheet_name)

            # Verificar si la hoja está en la estructura esperada
            if sheet_name in column_structure:
                cleaned_df = limpiar_hoja(df, column_structure[sheet_name])
                
                # Guardar la hoja procesada en un archivo temporal CSV
                temp_file_path = os.path.join(temp_folder, f"{sheet_name}_processed.csv")
                cleaned_df.to_csv(temp_file_path, index=False)
                print(f"Hoja '{sheet_name}' procesada y guardada en {temp_file_path}")
            else:
                print(f"Omitiendo la hoja '{sheet_name}' porque no tiene estructura conocida.")

        except Exception as e:
            print(f"Error al procesar la hoja '{sheet_name}': {e}")

def deduplicate_columns(df):
    """
    Deduplica nombres de columnas en el DataFrame.
    """
    df.columns = pd.io.parsers.ParserBase({'names': df.columns})._maybe_dedup_names(df.columns)
    return df

def flujo_completo(file_path, output_path):
    """
    Flujo completo para procesar el reporte TIC.
    Args:
        file_path (str): Ruta del archivo Excel.
        output_path (str): Carpeta donde se guardará el dataset final.
    """
    # Define las columnas esperadas
    column_structure = {
        '1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'No. ACCESOS FIJOS A INTERNET'],
        '2': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
        '3': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
        '4,1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,2': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,3': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,4': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '5': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS (Pesos)'],
        '6': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. ABONADOS'],
        '7': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
        '8': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TRÁFICO (MB)'],
        '9': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. SUSCRIPTORES'],
        '10': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
        '11': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'TRÁFICO (MB)'],
        '12': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'LÍNEAS EN SERVICIO', 'LÍNEAS PREPAGO', 'LÍNEAS POSPAGO', 'LÍNEAS ACTIVADAS', 'LÍNEAS RETIRADAS'],
        '13': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'PROVEEDOR DESTINO', 'TRÁFICO PREPAGO', 'TRÁFICO POSPAGO'],
        '14': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CONSUMO PREPAGO', 'CONSUMO POSPAGO', 'INGRESOS OPERACIONALES'],
        '15': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'CABECERA MUNICIPAL', 'CÓDIGO DANE', 'CENTRO POBLADO', 'COBERTURA 2G', 'COBERTURA 3G', 'COBERTURA HSPA+', 'HSPA+DC', 'COBERTURA_4G', 'COBERTURA_LTE', 'COBERTURA_5G'],
        '16': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE DEPARTAMENTO', 'DEPARTAMENTO', 'CÓDIGO DANE MUNICIPIO', 'MUNICIPIO', 'SEGMENTO', 'LÍNEAS EN SERVICIO'],
        '17': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS TRÁFICO LDIE', 'INGRESOS TRÁFICO LDIS', 'INGRESOS TELEFONIA FIJA']
    }

    # Detectar todas las columnas únicas
    all_columns = detect_all_columns(file_path, column_structure)

    # Procesar el reporte con limpieza y validación
    temp_folder = os.path.join(output_path, "temp_lotes")
    os.makedirs(temp_folder, exist_ok=True)

    excel_file = pd.ExcelFile(file_path)

    for sheet_name in excel_file.sheet_names:
        try:
            # Leer la hoja
            df = pd.read_excel(file_path, sheet_name=sheet_name)

            # Validar si está en la estructura esperada
            if sheet_name in column_structure:
                cleaned_df = limpiar_hoja(df, column_structure[sheet_name])

                # Eliminar encabezados duplicados (si aparecen en los datos)
                if (cleaned_df.columns == column_structure[sheet_name]).all():
                    cleaned_df = cleaned_df.loc[~(cleaned_df == cleaned_df.columns).all(axis=1)]

                # Guardar como archivo temporal
                temp_file_path = os.path.join(temp_folder, f"{sheet_name}_processed.csv")
                cleaned_df.to_csv(temp_file_path, index=False)
                print(f"Hoja '{sheet_name}' procesada y guardada en {temp_file_path}")
            else:
                print(f"Omitiendo la hoja '{sheet_name}' porque no está en la estructura conocida.")
        except Exception as e:
            print(f"Error al procesar la hoja '{sheet_name}': {e}")

    
    

    # Combinar los archivos temporales procesados
    combined_data_path = os.path.join(output_path, "combined_data_final.csv")
    temp_files = [os.path.join(temp_folder, f) for f in os.listdir(temp_folder) if f.endswith('_processed.csv')]

    # Inicializar el DataFrame combinado
    combined_df = None

    for temp_file in temp_files:
        # Leer el archivo temporal
        current_df = pd.read_csv(temp_file, dtype=str, low_memory=False)

        if combined_df is None:
            # Si es el primer archivo, inicializamos el DataFrame combinado
            combined_df = current_df
        else:
            # Realizar merge usando las columnas comunes como clave
            common_columns = ['AÑO', 'TRIMESTRE', 'PROVEEDOR']
            combined_df = pd.merge(combined_df, current_df, on=common_columns, how='outer', suffixes=('', '_dup'))

    # Guardar el DataFrame combinado en un archivo final
    combined_df.to_csv(combined_data_path, index=False, encoding='utf-8')
    print(f"Archivo combinado final guardado en {combined_data_path}")

    return combined_df
    
def debug_sheet_names(file_path, column_structure):
    excel_file = pd.ExcelFile(file_path)
    sheet_names = excel_file.sheet_names
    print(f"Nombres de hojas en el archivo: {sheet_names}")
    print(f"Nombres esperados en column_structure: {list(column_structure.keys())}")    
    
    


def flujo_completo_por_lotes(file_path, output_path, column_structure, chunk_size=500_000):
    """
    Flujo completo para procesar el reporte TIC y combinarlo en un dataset final usando chunks para evitar errores de memoria.
    Args:
        file_path (str): Ruta del archivo Excel.
        output_path (str): Carpeta donde se guardará el dataset final.
        chunk_size (int): Tamaño máximo de filas para procesar en cada lote.
    """
    
    temp_folder = os.path.join(output_path, "temp_lotes")
    os.makedirs(temp_folder, exist_ok=True)

    # Procesar cada hoja en archivos temporales
    process_report_con_hojas_4x(file_path, column_structure, temp_folder=temp_folder)

    # Obtener archivos temporales procesados
    temp_files = [os.path.join(temp_folder, f) for f in os.listdir(temp_folder) if f.endswith('_processed.csv')]

    # Inicializar el archivo de salida incremental
    combined_data_path = os.path.join(output_path, "combined_data_final.csv")
    header_written = False

    for temp_file in temp_files:
        # Leer el archivo temporal por chunks
        for chunk in pd.read_csv(temp_file, dtype=str, chunksize=chunk_size):
            if not header_written:
                chunk.to_csv(combined_data_path, mode='w', index=False, header=True)
                header_written = True
            else:
                chunk.to_csv(combined_data_path, mode='a', index=False, header=False)

    print(f"Archivo combinado final guardado en {combined_data_path}")
    return combined_data_path

def flujo_completo_optimizado(file_path, output_path, column_structure, batch_size=5):
    """
    Flujo completo para procesar y combinar reportes en bloques para evitar problemas de memoria.
    Args:
        file_path (str): Ruta del archivo Excel.
        output_path (str): Carpeta para el dataset final.
        batch_size (int): Número de archivos temporales para procesar en cada bloque.
    """
    # Paso 1: Procesar cada hoja y guardar como archivos temporales
    
    temp_folder = os.path.join(output_path, "temp_lotes")
    os.makedirs(temp_folder, exist_ok=True)
    process_report_con_hojas_4x(file_path, column_structure, temp_folder=temp_folder)

    # Paso 2: Combinar por lotes y guardar resultados intermedios
    temp_files = [os.path.join(temp_folder, f) for f in os.listdir(temp_folder) if f.endswith('_processed.csv')]
    intermediate_combined_path = os.path.join(output_path, "intermediate_combined.csv")

    # Inicializar DataFrame vacío para combinación por lotes
    with open(intermediate_combined_path, 'w', newline='', encoding='utf-8') as output_file:
        for i in range(0, len(temp_files), batch_size):
            batch_files = temp_files[i:i + batch_size]
            batch_df = pd.concat([pd.read_csv(f, dtype=str) for f in batch_files], axis=0, ignore_index=True)

            # Reorganizar columnas únicas y escribir en archivo intermedio
            if i == 0:
                all_columns = list(set(batch_df.columns))
                batch_df = batch_df.reindex(columns=all_columns, fill_value='')
                batch_df.to_csv(output_file, mode='w', index=False, header=True)
            else:
                batch_df = batch_df.reindex(columns=all_columns, fill_value='')
                batch_df.to_csv(output_file, mode='a', index=False, header=False)
            print(f"Lote {i // batch_size + 1} combinado y guardado temporalmente.")

    # Paso 3: Leer archivo combinado intermedio y guardar resultado final
    final_output_path = os.path.join(output_path, "combined_data_final.csv")
    final_df = pd.read_csv(intermediate_combined_path, dtype=str)
    final_df.to_csv(final_output_path, index=False, encoding='utf-8')
    print(f"Archivo combinado final guardado en {final_output_path}")
    return final_df


Inicio del programa

In [4]:
# ## Ejecución del Script

# Descargar y procesar el reporte
file_path = fetch_report(BASE_URL)  # Descarga y guarda el archivo en subcarpeta /reporteColombiaTic


Se encontró la sección 'Destacados'.
URL del artículo: https://colombiatic.mintic.gov.co/679/w3-article-397520.html
El archivo 'reporteColombiaTic\reporte_tic_segundo_trimestre_2024.xlsx' ya existe. No se realizará la descarga.


In [5]:
debug_sheet_names(file_path, column_structure)

Nombres de hojas en el archivo: ['PORTADA', 'CONTENIDO', '1', '2', '3', '4,1', '4,2', '4,3', '4,4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']
Nombres esperados en column_structure: ['1', '2', '3', '4,1', '4,2', '4,3', '4,4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17']


In [6]:
# Define las columnas esperadas
column_structure = {
        '1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'No. ACCESOS FIJOS A INTERNET'],
        '2': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
        '3': ['AÑO', 'TRIMESTRE', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'No. ACCESOS FIJOS A INTERNET', 'POBLACIÓN DANE', 'PENETRACIÓN'],
        '4,1': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,2': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,3': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '4,4': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'SEGMENTO', 'TECNOLOGÍA', 'VELOCIDAD BAJADA', 'VELOCIDAD SUBIDA', 'No. ACCESOS FIJOS A INTERNET'],
        '5': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS (Pesos)'],
        '6': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. ABONADOS'],
        '7': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
        '8': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TRÁFICO (MB)'],
        '9': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'TECNOLOGÍA', 'No. SUSCRIPTORES'],
        '10': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'SEGMENTO', 'TERMINAL', 'INGRESOS'],
        '11': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'TRÁFICO (MB)'],
        '12': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'LÍNEAS EN SERVICIO', 'LÍNEAS PREPAGO', 'LÍNEAS POSPAGO', 'LÍNEAS ACTIVADAS', 'LÍNEAS RETIRADAS'],
        '13': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'PROVEEDOR DESTINO', 'TRÁFICO PREPAGO', 'TRÁFICO POSPAGO'],
        '14': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CONSUMO PREPAGO', 'CONSUMO POSPAGO', 'INGRESOS OPERACIONALES'],
        '15': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE', 'DEPARTAMENTO', 'CÓDIGO DANE', 'MUNICIPIO', 'CABECERA MUNICIPAL', 'CÓDIGO DANE', 'CENTRO POBLADO', 'COBERTURA 2G', 'COBERTURA 3G', 'COBERTURA HSPA+', 'HSPA+DC', 'COBERTURA_4G', 'COBERTURA_LTE', 'COBERTURA_5G'],
        '16': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'CÓDIGO DANE DEPARTAMENTO', 'DEPARTAMENTO', 'CÓDIGO DANE MUNICIPIO', 'MUNICIPIO', 'SEGMENTO', 'LÍNEAS EN SERVICIO'],
        '17': ['AÑO', 'TRIMESTRE', 'PROVEEDOR', 'INGRESOS TRÁFICO LDIE', 'INGRESOS TRÁFICO LDIS', 'INGRESOS TELEFONIA FIJA']
    }

In [7]:
output_path = "reporteCombinado"  # Carpeta donde se guardará el dataset final
#all_columns = detect_all_columns(file_path, column_structure)
df_final=flujo_completo_optimizado(file_path, output_path, column_structure)

Omitiendo la hoja 'PORTADA' porque no tiene estructura conocida.
Omitiendo la hoja 'CONTENIDO' porque no tiene estructura conocida.
Hoja '1' procesada y guardada en reporteCombinado\temp_lotes\1_processed.csv
Hoja '2' procesada y guardada en reporteCombinado\temp_lotes\2_processed.csv
Hoja '3' procesada y guardada en reporteCombinado\temp_lotes\3_processed.csv
Hoja '4,1' procesada y guardada en reporteCombinado\temp_lotes\4,1_processed.csv
Hoja '4,2' procesada y guardada en reporteCombinado\temp_lotes\4,2_processed.csv
Hoja '4,3' procesada y guardada en reporteCombinado\temp_lotes\4,3_processed.csv
Hoja '4,4' procesada y guardada en reporteCombinado\temp_lotes\4,4_processed.csv
Hoja '5' procesada y guardada en reporteCombinado\temp_lotes\5_processed.csv
Hoja '6' procesada y guardada en reporteCombinado\temp_lotes\6_processed.csv
Hoja '7' procesada y guardada en reporteCombinado\temp_lotes\7_processed.csv
Hoja '8' procesada y guardada en reporteCombinado\temp_lotes\8_processed.csv
Hoja 

In [40]:
datasetFinal = pd.read_csv("reporteCombinado/combined_data_final.csv")   
datasetFinal.head()

C:\Users\Josvaldes\AppData\Local\Temp\ipykernel_34664\1606582176.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  datasetFinal = pd.read_csv("reporteCombinado/combined_data_final.csv")


,INGRESOS OPERACIONALES,LÍNEAS EN SERVICIO,LÍNEAS RETIRADAS,LÍNEAS PREPAGO,TRÁFICO (MB),LÍNEAS POSPAGO,SEGMENTO,INGRESOS,TERMINAL,LÍNEAS ACTIVADAS,TRÁFICO PREPAGO,PROVEEDOR,CONSUMO PREPAGO,PROVEEDOR DESTINO,TRÁFICO POSPAGO,AÑO,TRIMESTRE,CONSUMO POSPAGO
0,NaN,NaN,NaN,NaN,NaN,NaN,SEGMENTO,INGRESOS,TERMINAL,NaN,NaN,PROVEEDOR,NaN,NaN,NaN,AÑO,TRIMESTRE,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,676051722,TELÉFONO MÓVIL,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,7936553,DATA CARD,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,1241491,DATA CARD,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,5444337923,TELÉFONO MÓVIL,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN


In [41]:
datasetFinal

,INGRESOS OPERACIONALES,LÍNEAS EN SERVICIO,LÍNEAS RETIRADAS,LÍNEAS PREPAGO,TRÁFICO (MB),LÍNEAS POSPAGO,SEGMENTO,INGRESOS,TERMINAL,LÍNEAS ACTIVADAS,TRÁFICO PREPAGO,PROVEEDOR,CONSUMO PREPAGO,PROVEEDOR DESTINO,TRÁFICO POSPAGO,AÑO,TRIMESTRE,CONSUMO POSPAGO
0,NaN,NaN,NaN,NaN,NaN,NaN,SEGMENTO,INGRESOS,TERMINAL,NaN,NaN,PROVEEDOR,NaN,NaN,NaN,AÑO,TRIMESTRE,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,676051722,TELÉFONO MÓVIL,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,7936553,DATA CARD,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,1241491,DATA CARD,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,5444337923,TELÉFONO MÓVIL,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2269549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2269550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2269551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2269552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# Eliminar la primera fila del DataFrame
datasetFinal = datasetFinal.iloc[1:].reset_index(drop=True)

In [43]:
datasetFinal

,INGRESOS OPERACIONALES,LÍNEAS EN SERVICIO,LÍNEAS RETIRADAS,LÍNEAS PREPAGO,TRÁFICO (MB),LÍNEAS POSPAGO,SEGMENTO,INGRESOS,TERMINAL,LÍNEAS ACTIVADAS,TRÁFICO PREPAGO,PROVEEDOR,CONSUMO PREPAGO,PROVEEDOR DESTINO,TRÁFICO POSPAGO,AÑO,TRIMESTRE,CONSUMO POSPAGO
0,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,676051722,TELÉFONO MÓVIL,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,7936553,DATA CARD,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,1241491,DATA CARD,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,5444337923,TELÉFONO MÓVIL,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,19091681473,TELÉFONO MÓVIL,NaN,NaN,COLOMBIA MOVIL S.A ESP,NaN,NaN,NaN,2021,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2269548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2269549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2269550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2269551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Verificar los valores únicos nuevamente
print("Años únicos después de eliminar la primera fila:", datasetFinal['AÑO'].unique())
print("Trimestres únicos después de eliminar la primera fila:", datasetFinal['TRIMESTRE'].unique())
print("Proveedores únicos después de eliminar la primera fila:", datasetFinal['PROVEEDOR'].nunique())

Años únicos después de eliminar la primera fila: ['2021' '2022' '2023' '2024' 'AÑO' 2022 2023 2021 2024 nan]
Trimestres únicos después de eliminar la primera fila: ['4' '1' '2' '3' 'TRIMESTRE' 2 3 4 1 nan]
Proveedores únicos después de eliminar la primera fila: 1611


In [45]:
# Filtrar las filas válidas en la columna 'AÑO'
# Asegurarse de que 'AÑO' solo contenga años válidos
datasetFinal = datasetFinal[datasetFinal['AÑO'].astype(str).str.match(r'^\d{4}$')]

In [46]:
# Filtrar las filas válidas en la columna 'TRIMESTRE'
# Asegurarse de que 'TRIMESTRE' solo contenga valores de 1 a 4
datasetFinal = datasetFinal[datasetFinal['TRIMESTRE'].astype(str).str.match(r'^[1-4]$')]

# Verificar resultados
print("Años únicos después de limpiar:", datasetFinal['AÑO'].unique())
print("Trimestres únicos después de limpiar:", datasetFinal['TRIMESTRE'].unique())
print("Proveedores únicos después de limpiar:", datasetFinal['PROVEEDOR'].nunique())

Años únicos después de limpiar: ['2021' '2022' '2023' '2024' 2022 2023 2021 2024]
Trimestres únicos después de limpiar: ['4' '1' '2' '3' 2 3 4 1]
Proveedores únicos después de limpiar: 1610


In [47]:
# Convertir los valores en 'AÑO' y 'TRIMESTRE' a cadenas
datasetFinal['AÑO'] = datasetFinal['AÑO'].astype(str).str.strip()
datasetFinal['TRIMESTRE'] = datasetFinal['TRIMESTRE'].astype(str).str.strip()

# Filtrar valores válidos en 'AÑO' (solo años de 4 dígitos)
datasetFinal = datasetFinal[datasetFinal['AÑO'].str.match(r'^\d{4}$')]

# Filtrar valores válidos en 'TRIMESTRE' (solo trimestres 1-4)
datasetFinal = datasetFinal[datasetFinal['TRIMESTRE'].str.match(r'^[1-4]$')]

# Convertir los valores unificados nuevamente a enteros si es necesario
datasetFinal['AÑO'] = datasetFinal['AÑO'].astype(int)
datasetFinal['TRIMESTRE'] = datasetFinal['TRIMESTRE'].astype(int)

# Verificar resultados después de la limpieza
print("Años únicos después de unificar y limpiar:", datasetFinal['AÑO'].unique())
print("Trimestres únicos después de unificar y limpiar:", datasetFinal['TRIMESTRE'].unique())
print("Proveedores únicos después de unificar y limpiar:", datasetFinal['PROVEEDOR'].nunique())

Años únicos después de unificar y limpiar: [2021 2022 2023 2024]
Trimestres únicos después de unificar y limpiar: [4 1 2 3]
Proveedores únicos después de unificar y limpiar: 1610


In [48]:
print("Columnas en el dataset final:", datasetFinal.columns.tolist())

Columnas en el dataset final: ['INGRESOS OPERACIONALES', 'LÍNEAS EN SERVICIO', 'LÍNEAS RETIRADAS', 'LÍNEAS PREPAGO', 'TRÁFICO (MB)', 'LÍNEAS POSPAGO', 'SEGMENTO', 'INGRESOS', 'TERMINAL', 'LÍNEAS ACTIVADAS', 'TRÁFICO PREPAGO', 'PROVEEDOR', 'CONSUMO PREPAGO', 'PROVEEDOR DESTINO', 'TRÁFICO POSPAGO', 'AÑO', 'TRIMESTRE', 'CONSUMO POSPAGO']


In [49]:
datasetFinal

,INGRESOS OPERACIONALES,LÍNEAS EN SERVICIO,LÍNEAS RETIRADAS,LÍNEAS PREPAGO,TRÁFICO (MB),LÍNEAS POSPAGO,SEGMENTO,INGRESOS,TERMINAL,LÍNEAS ACTIVADAS,TRÁFICO PREPAGO,PROVEEDOR,CONSUMO PREPAGO,PROVEEDOR DESTINO,TRÁFICO POSPAGO,AÑO,TRIMESTRE,CONSUMO POSPAGO
0,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,676051722,TELÉFONO MÓVIL,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,7936553,DATA CARD,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,1241491,DATA CARD,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,5444337923,TELÉFONO MÓVIL,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,19091681473,TELÉFONO MÓVIL,NaN,NaN,COLOMBIA MOVIL S.A ESP,NaN,NaN,NaN,2021,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2269500,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,NaN,TELÉFONO MÓVIL,NaN,NaN,PARTNERS TELECOM COLOMBIA SAS,NaN,NaN,NaN,2024,2,NaN
2269501,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,NaN,TELÉFONO MÓVIL,NaN,NaN,TELECALL COLOMBIA SAS,NaN,NaN,NaN,2024,2,NaN
2269502,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,NaN,TELÉFONO MÓVIL,NaN,NaN,VIRGIN MOBILE COLOMBIA S.A.S.,NaN,NaN,NaN,2024,2,NaN
2269503,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,NaN,TELÉFONO MÓVIL,NaN,NaN,VIRGIN MOBILE COLOMBIA S.A.S.,NaN,NaN,NaN,2024,2,NaN


1. Identificar valores nulos
Antes de manejar los valores nulos, identifica dónde están presentes y cuáles son las columnas más afectadas:

In [50]:
# Mostrar el resumen de valores nulos por columna
missing_summary = datasetFinal.isnull().sum()
print("Valores nulos por columna:\n", missing_summary[missing_summary > 0])

Valores nulos por columna:
 INGRESOS OPERACIONALES    2269350
LÍNEAS EN SERVICIO        2220868
LÍNEAS RETIRADAS          2269351
LÍNEAS PREPAGO            2269351
TRÁFICO (MB)              2269234
LÍNEAS POSPAGO            2269351
SEGMENTO                   163054
INGRESOS                  2269008
TERMINAL                  2267847
LÍNEAS ACTIVADAS          2269351
TRÁFICO PREPAGO           2267397
PROVEEDOR                   12658
CONSUMO PREPAGO           2269350
PROVEEDOR DESTINO         2267397
TRÁFICO POSPAGO           2267397
CONSUMO POSPAGO           2269350
dtype: int64


2. Analizar estadísticas descriptivas
Genera un resumen estadístico para las columnas numéricas y verifica posibles outliers:

In [51]:
# Estadísticas descriptivas para columnas numéricas
print(datasetFinal.describe())


                AÑO     TRIMESTRE
count  2.269486e+06  2.269486e+06
mean   2.022710e+03  2.400622e+00
std    8.291372e-01  1.136601e+00
min    2.021000e+03  1.000000e+00
25%    2.022000e+03  1.000000e+00
50%    2.023000e+03  2.000000e+00
75%    2.023000e+03  3.000000e+00
max    2.024000e+03  4.000000e+00


In [52]:
# Estadísticas para columnas categóricas
print(datasetFinal.describe(include=['object']))

       INGRESOS OPERACIONALES  LÍNEAS EN SERVICIO LÍNEAS RETIRADAS   
count                     136               48618              135  \
unique                    126                7120              126   
top                         0                   1                0   
freq                       11                5489                9   

       LÍNEAS PREPAGO TRÁFICO (MB) LÍNEAS POSPAGO     SEGMENTO INGRESOS   
count             135          252            135      2106432      478  \
unique            130          228             60           14      449   
top                 0            0              0  CORPORATIVO        0   
freq                6           20             76       560757       25   

              TERMINAL LÍNEAS ACTIVADAS TRÁFICO PREPAGO   
count             1639              135            2089  \
unique               2              129            1681   
top     TELÉFONO MÓVIL                0               0   
freq              1045               

3. Detectar y manejar duplicados
Identifica y elimina filas duplicadas en el dataset:

In [53]:
# Identificar filas duplicadas
duplicates = datasetFinal.duplicated()
print(f"Número de filas duplicadas: {duplicates.sum()}")

# Eliminar filas duplicadas
datasetFinal = datasetFinal.drop_duplicates()

Número de filas duplicadas: 2195492


4. Validar integridad de las columnas clave
Revisa si las columnas clave (AÑO, TRIMESTRE, PROVEEDOR) tienen valores inesperados o vacíos:

In [54]:
# Validar valores únicos en columnas clave
print("Años únicos:", datasetFinal['AÑO'].unique())
print("Trimestres únicos:", datasetFinal['TRIMESTRE'].unique())
print("Proveedores únicos:", datasetFinal['PROVEEDOR'].nunique())

# Filtrar filas donde columnas clave están vacías
datasetFinal = datasetFinal.dropna(subset=['AÑO', 'TRIMESTRE', 'PROVEEDOR'])

Años únicos: [2021 2022 2023 2024]
Trimestres únicos: [4 1 2 3]
Proveedores únicos: 1610


5. Unificar nombres de proveedores
Si encuentras inconsistencias en nombres de proveedores (por ejemplo, "AVANTEL" y "Avantel S.A.S"), normaliza los valores:

In [60]:
# Convertir a mayúsculas y eliminar espacios
datasetFinal['PROVEEDOR'] = datasetFinal['PROVEEDOR'].str.upper().str.strip()

# Correcciones manuales si es necesario
datasetFinal['PROVEEDOR'] = datasetFinal['PROVEEDOR'].replace({
    'AVANTEL S.A.S.': 'AVANTEL S.A.S',
    'CLARO TELECOM': 'CLARO'
})

6. Detectar outliers
Utiliza el rango intercuartílico (IQR) para detectar valores atípicos en las columnas numéricas:

In [61]:
# Convertir la columna 'INGRESOS' a formato numérico
datasetFinal['INGRESOS'] 

0          6.760517e+08
1          7.936553e+06
2          1.241491e+06
3          5.444338e+09
4          1.909168e+10
               ...     
2269497             NaN
2269498             NaN
2269499             NaN
2269501             NaN
2269502             NaN
Name: INGRESOS, Length: 73983, dtype: float64

In [62]:
datasetFinal['INGRESOS']  = pd.to_numeric(datasetFinal['INGRESOS'], errors='coerce')

In [63]:
# Detectar outliers en 'INGRESOS'
Q1 = datasetFinal['INGRESOS'].quantile(0.25)
Q3 = datasetFinal['INGRESOS'].quantile(0.75)
IQR = Q3 - Q1
outliers = datasetFinal[(datasetFinal['INGRESOS'] < (Q1 - 1.5 * IQR)) | 
                        (datasetFinal['INGRESOS'] > (Q3 + 1.5 * IQR))]

print("Número de outliers en INGRESOS:", len(outliers))

Número de outliers en INGRESOS: 90


In [64]:
# Verificar valores faltantes después de la conversión
print("Valores faltantes en 'INGRESOS' después de la conversión:", datasetFinal['INGRESOS'].isnull().sum())

# Filtrar filas con valores válidos en 'INGRESOS'
datasetFinal = datasetFinal[datasetFinal['INGRESOS'].notnull()]

# Detectar outliers usando el rango intercuartílico (IQR)
Q1 = datasetFinal['INGRESOS'].quantile(0.25)
Q3 = datasetFinal['INGRESOS'].quantile(0.75)
IQR = Q3 - Q1

# Límites para detectar outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrar outliers
outliers = datasetFinal[(datasetFinal['INGRESOS'] < lower_bound) | (datasetFinal['INGRESOS'] > upper_bound)]
print(f"Número de outliers detectados en 'INGRESOS': {len(outliers)}")

# Opcional: Eliminar outliers
datasetFinal = datasetFinal[(datasetFinal['INGRESOS'] >= lower_bound) & (datasetFinal['INGRESOS'] <= upper_bound)]

# Verificar estadísticas después de limpiar
print("Estadísticas de 'INGRESOS' después de limpiar:")
print(datasetFinal['INGRESOS'].describe())

Valores faltantes en 'INGRESOS' después de la conversión: 73505
Número de outliers detectados en 'INGRESOS': 90
Estadísticas de 'INGRESOS' después de limpiar:
count    3.880000e+02
mean     3.342586e+09
std      6.612175e+09
min      0.000000e+00
25%      5.275258e+07
50%      4.012013e+08
75%      1.824508e+09
max      3.782608e+10
Name: INGRESOS, dtype: float64


7. Validar relaciones entre columnas
Revisa si las relaciones entre columnas tienen sentido (por ejemplo, LÍNEAS POSPAGO + LÍNEAS PREPAGO debería igualar LÍNEAS EN SERVICIO):

In [65]:
# Validar relaciones lógicas
datasetFinal['TOTAL_LÍNEAS'] = datasetFinal['LÍNEAS PREPAGO'] + datasetFinal['LÍNEAS POSPAGO']
inconsistencias = datasetFinal[datasetFinal['TOTAL_LÍNEAS'] != datasetFinal['LÍNEAS EN SERVICIO']]
print(f"Registros con inconsistencias en líneas: {len(inconsistencias)}")

Registros con inconsistencias en líneas: 388


8. Exportar el dataset limpio
Guarda el dataset final para análisis o procesamiento adicional:

In [66]:
# Guardar el dataset depurado
datasetFinal.to_csv('reporteCombinado/cleaned_data_final.csv', index=False, encoding='utf-8')
print("Dataset final guardado en reporteCombinado/cleaned_data_final.csv")

Dataset final guardado en reporteCombinado/cleaned_data_final.csv


In [69]:
datasetFinal

,INGRESOS OPERACIONALES,LÍNEAS EN SERVICIO,LÍNEAS RETIRADAS,LÍNEAS PREPAGO,TRÁFICO (MB),LÍNEAS POSPAGO,SEGMENTO,INGRESOS,TERMINAL,LÍNEAS ACTIVADAS,TRÁFICO PREPAGO,PROVEEDOR,CONSUMO PREPAGO,PROVEEDOR DESTINO,TRÁFICO POSPAGO,AÑO,TRIMESTRE,CONSUMO POSPAGO,TOTAL_LÍNEAS
0,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,6.760517e+08,TELÉFONO MÓVIL,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,7.936553e+06,DATA CARD,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,1.241491e+06,DATA CARD,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,PERSONAS,5.444338e+09,TELÉFONO MÓVIL,NaN,NaN,AVANTEL S.A.S,NaN,NaN,NaN,2021,4,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,EMPRESAS,1.909168e+10,TELÉFONO MÓVIL,NaN,NaN,COLOMBIA MOVIL S.A ESP,NaN,NaN,NaN,2021,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2268759,NaN,NaN,NaN,NaN,NaN,NaN,PREPAGO,1.367506e+09,TELÉFONO MÓVIL,NaN,NaN,LOGISTICA FLASH COLOMBIA S.A.S,NaN,NaN,NaN,2024,2,NaN,NaN
2268760,NaN,NaN,NaN,NaN,NaN,NaN,PREPAGO,5.098238e+07,TELÉFONO MÓVIL,NaN,NaN,LOV TELECOMUNICACIONES SAS,NaN,NaN,NaN,2024,2,NaN,NaN
2268761,NaN,NaN,NaN,NaN,NaN,NaN,PREPAGO,2.384082e+10,TELÉFONO MÓVIL,NaN,NaN,PARTNERS TELECOM COLOMBIA SAS,NaN,NaN,NaN,2024,2,NaN,NaN
2268762,NaN,NaN,NaN,NaN,NaN,NaN,PREPAGO,3.625667e+08,TELÉFONO MÓVIL,NaN,NaN,SUMA MOVIL S.A.S.,NaN,NaN,NaN,2024,2,NaN,NaN


In [67]:
# ## Ejecución de Validación y Análisis

# Realizar análisis de columnas clave
analyze_columns(datasetFinal)




Valores únicos en 'AÑO': [2021 2022 2023 2024]

Valores únicos en 'TRIMESTRE': [4 1 2 3]

Valores únicos en 'PROVEEDOR': ['AVANTEL S.A.S' 'COLOMBIA MOVIL  S.A ESP'
 'COLOMBIA TELECOMUNICACIONES S.A. E.S.P.'
 'COMUNICACION CELULAR S A COMCEL S A'
 'EMPRESA DE TELECOMUNICACIONES DE BOGOTA S.A. ESP'
 'PARTNERS TELECOM COLOMBIA SAS' 'SETROC MOBILE GROUP SAS'
 'VIRGIN MOBILE COLOMBIA S.A.S.' 'INTELNEXT SAS' 'INSIDENET SAS']


In [68]:
# Manejar valores nulos
#combined_df = handle_missing_values(datasetFinal)

# Generar estadísticas descriptivas
generate_statistics(datasetFinal)

# Guardar la versión final del DataFrame validado
save_final_data(datasetFinal)


Estadísticas descriptivas de columnas numéricas:
            INGRESOS          AÑO   TRIMESTRE
count  3.880000e+02   388.000000  388.000000
mean   3.342586e+09  2022.600515    2.443299
std    6.612175e+09     0.884959    1.147689
min    0.000000e+00  2021.000000    1.000000
25%    5.275258e+07  2022.000000    1.000000
50%    4.012013e+08  2023.000000    2.000000
75%    1.824508e+09  2023.000000    4.000000
max    3.782608e+10  2024.000000    4.000000
DataFrame final validado guardado en: reporteCombinado\data_reporte_combined_validated.xlsx


Combinación manual

In [75]:
hoja1 = pd.read_csv("reporteCombinado/temp_lotes/1_processed.csv")

In [76]:
hoja1.head()

,AÑO,TRIMESTRE,PROVEEDOR,No. ACCESOS FIJOS A INTERNET
0,AÑO,TRIMESTRE,PROVEEDOR,No. ACCESOS FIJOS A INTERNET
1,2021,4,INTER CONEXIONES R Y L S.A.S,24
2,2021,4,ASIS OBRAS Y SERVICIOS S.A.S,143
3,2021,4,WIFI ALTERNATIVO VALLE S.A.S,218
4,2021,4,MAXTECHNOLOGY COMUNICACIONES S.A.S,171


In [81]:
# Eliminar la primera fila del DataFrame
hoja1 = hoja1.iloc[1:].reset_index(drop=True)

In [82]:
hoja1.head()

,AÑO,TRIMESTRE,PROVEEDOR,No. ACCESOS FIJOS A INTERNET
0,2021,4,INTER CONEXIONES R Y L S.A.S,24
1,2021,4,ASIS OBRAS Y SERVICIOS S.A.S,143
2,2021,4,WIFI ALTERNATIVO VALLE S.A.S,218
3,2021,4,MAXTECHNOLOGY COMUNICACIONES S.A.S,171
4,2021,4,ANTNET COMUNICACIONES S.A.S.,532


In [83]:
hoja1.shape

(10652, 4)

In [84]:
hoja2 = pd.read_csv("reporteCombinado/temp_lotes/2_processed.csv")

In [86]:
hoja2.head()

,AÑO,TRIMESTRE,CÓDIGO DANE,DEPARTAMENTO,No. ACCESOS FIJOS A INTERNET,POBLACIÓN DANE,PENETRACIÓN
0,AÑO,TRIMESTRE,CÓDIGO DANE,DEPARTAMENTO,No. ACCESOS FIJOS A INTERNET,POBLACIÓN DANE,PENETRACIÓN
1,2021,4,91,AMAZONAS,1251,82171,0.015224349222961873
2,2021,4,05,ANTIOQUIA,1513196,6726219,0.22496977871223045
3,2021,4,81,ARAUCA,25826,300637,0.08590426328096674
4,2021,4,88,"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANT...",4176,62159,0.06718254798178865


In [87]:
# Eliminar la primera fila del DataFrame
hoja2 = hoja2.iloc[1:].reset_index(drop=True)

In [88]:
hoja2.head()

,AÑO,TRIMESTRE,CÓDIGO DANE,DEPARTAMENTO,No. ACCESOS FIJOS A INTERNET,POBLACIÓN DANE,PENETRACIÓN
0,2021,4,91,AMAZONAS,1251,82171,0.015224349222961873
1,2021,4,05,ANTIOQUIA,1513196,6726219,0.22496977871223045
2,2021,4,81,ARAUCA,25826,300637,0.08590426328096674
3,2021,4,88,"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANT...",4176,62159,0.06718254798178865
4,2021,4,08,ATLÁNTICO,427100,2741002,0.15581893044952175


In [89]:
hoja2.shape

(363, 7)